In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

from scipy import stats as sps
from scipy.interpolate import interp1d

from IPython.display import clear_output

In [85]:
import copy, math

In [86]:
import plotly.express as px

In [87]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [88]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
#import plotly.graph_objects as go
init_notebook_mode(connected=True)

# Define all keys

In [6]:
state_key = {
'EC':'Eastern Cape',
'FS':'Free State',
'GP':'Gauteng',
'KZN':'Kwazulu Natal',
'LP':'Limpopo',
'MP':'Mpumalanga',
'NC':'Northern Cape',
'NW':'North-West',
'WC':'Western Cape'
}
state_filter = list(state_key.keys())
state_filter

['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

In [7]:
state_labels = list(state_key.values())

In [8]:
state_filter_i = copy.deepcopy(state_filter)
state_filter_i.append('Date')
state_filter_i

['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC', 'Date']

In [9]:
state_filter

['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

In [11]:
# Mid-year 2019 Estimates
# https://www.statssa.gov.za/publications/P0302/P03022019.pdf
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/district_data/za_province_pop.csv'
province_pops = pd.read_csv(url, header=None, names=['Province','Pop'])
province_pops

,Province,Pop
0,Gauteng,15176115
1,KwaZulu-Natal,11289086
2,Western Cape,6844272
3,Eastern Cape,6712276
4,Limpopo,5982584
5,Mpumalanga,4592187
6,Northwest,4027160
7,Free State,2887465
8,Northern Cape,1263875


In [12]:
country_pop = province_pops['Pop'].sum()
country_pop

58775020

In [13]:
province_pops.iloc[0]['Pop']

15176115

# Get Latest Province Data

In [14]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv'
states_all_i = pd.read_csv(url, parse_dates=['date'], dayfirst=True, squeeze=True, index_col=0)
states_all_i.tail()

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-06-17,20200617,11919.0,608.0,14620.0,4238.0,416.0,391.0,211.0,1402.0,46555.0,52.0,80412,https://twitter.com/nicd_sa/status/12733358586...
2020-06-18,20200618,12639.0,649.0,15898.0,4360.0,489.0,461.0,214.0,1606.0,47522.0,52.0,83890,https://twitter.com/nicd_sa/status/12737017191...
2020-06-19,20200619,13506.0,658.0,17261.0,4548.0,503.0,505.0,218.0,1758.0,48706.0,52.0,87715,https://twitter.com/nicd_sa/status/12740622985...
2020-06-20,20200620,14721.0,700.0,18978.0,4806.0,521.0,544.0,232.0,1957.0,50173.0,49.0,92681,https://twitter.com/nicd_sa/status/12744018338...
2020-06-21,20200621,15751.0,733.0,20822.0,5030.0,564.0,573.0,237.0,2148.0,51441.0,3.0,97302,https://twitter.com/nicd_sa/status/12747874821...


In [15]:
states_all = states_all_i.copy()
states_all = states_all.reset_index()
states_all = states_all.rename(columns={'date':'Date'})
states_all.tail()

,Date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
102,2020-06-17,20200617,11919.0,608.0,14620.0,4238.0,416.0,391.0,211.0,1402.0,46555.0,52.0,80412,https://twitter.com/nicd_sa/status/12733358586...
103,2020-06-18,20200618,12639.0,649.0,15898.0,4360.0,489.0,461.0,214.0,1606.0,47522.0,52.0,83890,https://twitter.com/nicd_sa/status/12737017191...
104,2020-06-19,20200619,13506.0,658.0,17261.0,4548.0,503.0,505.0,218.0,1758.0,48706.0,52.0,87715,https://twitter.com/nicd_sa/status/12740622985...
105,2020-06-20,20200620,14721.0,700.0,18978.0,4806.0,521.0,544.0,232.0,1957.0,50173.0,49.0,92681,https://twitter.com/nicd_sa/status/12744018338...
106,2020-06-21,20200621,15751.0,733.0,20822.0,5030.0,564.0,573.0,237.0,2148.0,51441.0,3.0,97302,https://twitter.com/nicd_sa/status/12747874821...


In [16]:
state_plot = states_all[state_filter_i]
state_plot

,EC,FS,GP,KZN,LP,MP,NC,NW,WC,Date
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2020-03-05
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2020-03-07
2,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2020-03-08
3,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2020-03-09
4,0.0,0.0,5.0,7.0,0.0,0.0,0.0,0.0,1.0,2020-03-11
...,...,...,...,...,...,...,...,...,...,...
102,11919.0,608.0,14620.0,4238.0,416.0,391.0,211.0,1402.0,46555.0,2020-06-17
103,12639.0,649.0,15898.0,4360.0,489.0,461.0,214.0,1606.0,47522.0,2020-06-18
104,13506.0,658.0,17261.0,4548.0,503.0,505.0,218.0,1758.0,48706.0,2020-06-19
105,14721.0,700.0,18978.0,4806.0,521.0,544.0,232.0,1957.0,50173.0,2020-06-20


In [17]:
state_plotly = state_plot.melt(id_vars='Date', var_name='Province', value_name='Cases')
state_plotly

,Date,Province,Cases
0,2020-03-05,EC,0.0
1,2020-03-07,EC,0.0
2,2020-03-08,EC,0.0
3,2020-03-09,EC,0.0
4,2020-03-11,EC,0.0
...,...,...,...
958,2020-06-17,WC,46555.0
959,2020-06-18,WC,47522.0
960,2020-06-19,WC,48706.0
961,2020-06-20,WC,50173.0


In [18]:
#https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express
colour_series = px.colors.qualitative.Vivid
colour_series

['rgb(229, 134, 6)',
 'rgb(93, 105, 177)',
 'rgb(82, 188, 163)',
 'rgb(153, 201, 69)',
 'rgb(204, 97, 176)',
 'rgb(36, 121, 108)',
 'rgb(218, 165, 27)',
 'rgb(47, 138, 196)',
 'rgb(118, 78, 159)',
 'rgb(237, 100, 90)',
 'rgb(165, 170, 153)']

In [19]:
fig = px.bar(state_plotly, title='Total Cases Per Province', x='Date', y='Cases', color='Province',
             barmode='relative', color_discrete_sequence=colour_series)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x")
fig.show()

In [20]:
states_all['Actual Data'] = states_all['total'].diff()

In [21]:
smoothed = states_all['Actual Data'].rolling(7,
    win_type='gaussian',
    min_periods=1,
    center=True).mean(std=2).round()

idx_start = np.searchsorted(smoothed, 25)

smoothed = smoothed.iloc[idx_start:]
states_all['Smoothed Data'] = smoothed

In [22]:
daily = states_all[['Date','Actual Data','Smoothed Data']]

In [23]:
daily_plotly = daily.melt(id_vars='Date', var_name='Range', value_name='Daily Cases')

In [24]:
fig1 = px.line(daily_plotly, title='Daily Case Increase for South Africa',
        x='Date', y='Daily Cases', color='Range', line_shape='spline')
fig1.update_traces(hovertemplate=None)
fig1.update_layout(hovermode="x")
fig1.show()

In [25]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_deaths.csv'
states_all_deaths = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True,index_col=0).sort_index()
states_all_deaths.tail()

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-06-17,20200617,251,9,87,73,4,1,1,5,1243,0,1674,https://twitter.com/nicd_sa/status/12733358586...
2020-06-18,20200618,257,9,87,73,4,1,1,5,1300,0,1737,https://sacoronavirus.co.za/2020/06/18/update-...
2020-06-19,20200619,257,9,114,80,4,1,1,5,1360,0,1831,https://twitter.com/nicd_sa/status/12740622985...
2020-06-20,20200620,268,9,114,83,4,1,1,5,1392,0,1877,https://twitter.com/nicd_sa/status/12744018338...
2020-06-21,20200621,285,9,122,84,4,1,1,5,1419,0,1930,https://twitter.com/nicd_sa/status/12747874821...


In [26]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_recoveries.csv'
states_all_recover = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True,index_col=0).sort_index()
states_all_recover.tail()

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-06-17,20200617,5845,222,3716,2133,218,146,52,175,31824,0,44331,https://twitter.com/nicd_sa/status/12733358586...
2020-06-18,20200618,6331,235,3756,2133,229,156,81,175,31824,0,44920,https://sacoronavirus.co.za/2020/06/18/update-...
2020-06-19,20200619,6331,235,4001,2527,236,156,97,175,34067,0,47825,https://twitter.com/nicd_sa/status/12740622985...
2020-06-20,20200620,7470,236,4273,2527,236,156,97,175,35156,0,50326,https://twitter.com/nicd_sa/status/12744018338...
2020-06-21,20200621,7738,264,4673,2594,246,176,110,248,35559,0,51608,https://twitter.com/nicd_sa/status/12747874821...


In [27]:
cases_series = pd.Series(states_all_i['total'].values, index=states_all_i.index.values, name='Cases')
cases_series

2020-03-05        1
2020-03-07        2
2020-03-08        3
2020-03-09        7
2020-03-11       13
              ...  
2020-06-17    80412
2020-06-18    83890
2020-06-19    87715
2020-06-20    92681
2020-06-21    97302
Name: Cases, Length: 107, dtype: int64

In [28]:
deaths_series = pd.Series(states_all_deaths['total'].values, index=states_all_deaths.index, name='Deaths')
recover_series = pd.Series(states_all_recover['total'].values, index=states_all_recover.index, name='Recovered')

In [29]:
states_combine = pd.concat([cases_series, recover_series, deaths_series], axis=1)
states_combine

,Cases,Recovered,Deaths
2020-03-05,1,NaN,NaN
2020-03-07,2,NaN,NaN
2020-03-08,3,NaN,NaN
2020-03-09,7,NaN,NaN
2020-03-11,13,NaN,NaN
...,...,...,...
2020-06-17,80412,44331.0,1674.0
2020-06-18,83890,44920.0,1737.0
2020-06-19,87715,47825.0,1831.0
2020-06-20,92681,50326.0,1877.0


In [30]:
states_master = states_combine.ffill(axis=0)

In [31]:
states_changed = states_master[['Recovered','Deaths']].sum(axis=1)

In [32]:
active_all = states_master['Cases'].sub(states_changed)
active_all

2020-03-05        1.0
2020-03-07        2.0
2020-03-08        3.0
2020-03-09        7.0
2020-03-11       13.0
               ...   
2020-06-17    34407.0
2020-06-18    37233.0
2020-06-19    38059.0
2020-06-20    40478.0
2020-06-21    43764.0
Length: 107, dtype: float64

In [33]:
states_master['Active'] = active_all

In [34]:
states_master

,Cases,Recovered,Deaths,Active
2020-03-05,1,NaN,NaN,1.0
2020-03-07,2,NaN,NaN,2.0
2020-03-08,3,NaN,NaN,3.0
2020-03-09,7,NaN,NaN,7.0
2020-03-11,13,NaN,NaN,13.0
...,...,...,...,...
2020-06-17,80412,44331.0,1674.0,34407.0
2020-06-18,83890,44920.0,1737.0,37233.0
2020-06-19,87715,47825.0,1831.0,38059.0
2020-06-20,92681,50326.0,1877.0,40478.0


In [35]:
states_wide = states_master.reset_index()
states_wide = states_wide.rename(columns={'index':'Date'})
states_wide

,Date,Cases,Recovered,Deaths,Active
0,2020-03-05,1,NaN,NaN,1.0
1,2020-03-07,2,NaN,NaN,2.0
2,2020-03-08,3,NaN,NaN,3.0
3,2020-03-09,7,NaN,NaN,7.0
4,2020-03-11,13,NaN,NaN,13.0
...,...,...,...,...,...
102,2020-06-17,80412,44331.0,1674.0,34407.0
103,2020-06-18,83890,44920.0,1737.0,37233.0
104,2020-06-19,87715,47825.0,1831.0,38059.0
105,2020-06-20,92681,50326.0,1877.0,40478.0


In [36]:
state_wide_plotly = states_wide.melt(id_vars='Date', var_name='Data', value_name='Count')
state_wide_plotly

,Date,Data,Count
0,2020-03-05,Cases,1.0
1,2020-03-07,Cases,2.0
2,2020-03-08,Cases,3.0
3,2020-03-09,Cases,7.0
4,2020-03-11,Cases,13.0
...,...,...,...
423,2020-06-17,Active,34407.0
424,2020-06-18,Active,37233.0
425,2020-06-19,Active,38059.0
426,2020-06-20,Active,40478.0


In [37]:
fig2 = px.line(state_wide_plotly, x='Date', y='Count', color='Data',
              title='Covid-19 Data for South Africa', line_shape='spline')
fig2.update_traces(hovertemplate=None)
fig2.update_layout(hovermode="x")
fig2.show()

# Summmary Data

In [38]:
latestcases = states_wide.iloc[-1,:]
latest = [latestcases['Cases'], latestcases['Recovered'], latestcases['Deaths'], latestcases['Active']]
summary = [int(num) for num in latest]
summary[0]

97302

## Testing data

In [39]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_testing.csv'
states_all_tests = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True,index_col=0).sort_index()
states_all_tests

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-04-27,20200427,14054,6317,61830,28265,3691,4385,1314,2544,33335,12908,168643,ministerial briefing 26-04-2020
2020-05-03,20200503,24835,12634,85134,45955,5626,6691,1951,3565,53967,17183,257541,https://www.nicd.ac.za/wp-content/uploads/2020...
2020-05-09,20200509,34032,17231,111630,61801,7783,9823,3092,5213,70038,20693,341336,https://www.nicd.ac.za/wp-content/uploads/2020...
2020-05-13,20200513,38122,19265,127030,67853,8239,11414,3683,5812,82865,22069,386352,https://twitter.com/bandilemasuku/status/12609...
2020-05-20,20200520,51888,24415,166394,86448,10579,14964,4776,8830,112612,25955,506681,https://twitter.com/GautengProvince/status/126...
2020-05-28,20200528,66013,34760,215959,109643,14365,18629,6061,11214,148174,30905,655723,https://twitter.com/GautengProvince/status/126...


In [40]:
states_tests = states_all_tests[state_filter]

# Evaluate district data

In [41]:
districts_gp = {
'date':'date',
'Ekurhuleni\tCases':'Ekurhuleni',
'Johannesburg\tCases':'Johannesburg',
'Sedibeng\tCases':'Sedibeng',
'Tshwane\tCases':'Tshwane',
'West Rand\tCases':'West Rand',
'West Rand\tCases':'West Rand',
'GP Unallocated\tCases':'Unknown'
}
districts = districts_gp.keys()

In [42]:
file_name = 'provincial_' + 'gp' + '_cumulative.csv'  
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/district_data/' + file_name
states_district = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True).sort_index()

states_district_filter = states_district[districts]
col_tol = states_district_filter.sum(axis=1, numeric_only=True)
pd.options.mode.chained_assignment = None
states_district_filter['Total'] = col_tol

#states = states_filter_wp
states_district_filter.tail()

,date,Ekurhuleni\tCases,Johannesburg\tCases,Sedibeng\tCases,Tshwane\tCases,West Rand\tCases,GP Unallocated\tCases,Total
67,2020-06-17,2970.0,7229.0,434.0,2116.0,1061.0,810.0,14620.0
68,2020-06-18,3192.0,7899.0,497.0,2357.0,1130.0,823.0,15898.0
69,2020-06-19,3492.0,8575.0,555.0,2572.0,1225.0,842.0,17261.0
70,2020-06-20,3830.0,9322.0,606.0,2891.0,1330.0,999.0,18978.0
71,2020-06-21,4168.0,10320.0,675.0,3166.0,1439.0,1054.0,20822.0


# Rt Data Import

In [43]:
#dsfsi
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/calc/calculated_rt_sa_provincial_cumulative.csv'
states_all_rt = pd.read_csv(url, parse_dates=['date'], dayfirst=True, squeeze=True)
states_all_rt = states_all_rt.rename(columns={'date':'Date'})
states_all_rt = states_all_rt.rename(columns={'ML':'Rt'})
states_all_rt = states_all_rt.rename(columns={'state':'Province'})
states_all_rt

,Province,Date,Rt,High_90,Low_90
0,EC,2020-04-11,3.84,6.34,0.78
1,EC,2020-04-12,3.35,5.14,1.27
2,EC,2020-04-13,3.13,4.52,1.55
3,EC,2020-04-14,2.72,3.87,1.42
4,EC,2020-04-15,2.29,3.28,1.14
...,...,...,...,...,...
525,Total RSA,2020-06-17,1.13,1.30,0.95
526,Total RSA,2020-06-18,1.27,1.42,1.07
527,Total RSA,2020-06-19,1.36,1.51,1.17
528,Total RSA,2020-06-20,1.38,1.53,1.20


In [44]:
#state_single = states_rt.filter(like='Total RSA', axis=0) # for index data
state_single = states_all_rt.query("Province == 'Total RSA'")
state_single

,Province,Date,Rt,High_90,Low_90
435,Total RSA,2020-03-19,2.11,3.70,0.22
436,Total RSA,2020-03-20,2.50,3.73,1.09
437,Total RSA,2020-03-21,2.75,3.71,1.67
438,Total RSA,2020-03-22,2.87,3.67,1.99
439,Total RSA,2020-03-23,2.97,3.64,2.20
...,...,...,...,...,...
525,Total RSA,2020-06-17,1.13,1.30,0.95
526,Total RSA,2020-06-18,1.27,1.42,1.07
527,Total RSA,2020-06-19,1.36,1.51,1.17
528,Total RSA,2020-06-20,1.38,1.53,1.20


In [45]:
latestresult = state_single.iloc[-1,:]
rt = round(latestresult['Rt'], 3)
latestrt = '%.2f'%rt
latestrt

'1.24'

In [46]:
rt

1.24

In [47]:
d = latestresult['Date']
latestd = d.strftime("%d %B %Y")
latestd

'21 June 2020'

In [48]:
state_single["e_plus"] = state_single['High_90'].sub(state_single['Rt'])
state_single["e_minus"] = state_single['Rt'].sub(state_single['Low_90'])

In [49]:
fig3 = px.line(state_single, x='Date', y='Rt', color='Province',
              error_y='e_plus', error_y_minus='e_minus',
              title='Rt for Covid-19 in South Africa', line_shape='spline')
fig3.update_traces(hovertemplate=None)
fig3.update_layout(hovermode="x")
fig3['data'][0]['error_y']['color'] = 'lightblue'

fig3.show()

### Old grid subplots

In [50]:
states_rt = states_all_rt.query("Province != 'Total RSA'")
states_rt

,Province,Date,Rt,High_90,Low_90
0,EC,2020-04-11,3.84,6.34,0.78
1,EC,2020-04-12,3.35,5.14,1.27
2,EC,2020-04-13,3.13,4.52,1.55
3,EC,2020-04-14,2.72,3.87,1.42
4,EC,2020-04-15,2.29,3.28,1.14
...,...,...,...,...,...
430,WC,2020-06-17,0.54,0.75,0.28
431,WC,2020-06-18,0.59,0.81,0.33
432,WC,2020-06-19,0.80,1.02,0.54
433,WC,2020-06-20,1.00,1.22,0.74


In [51]:
grid_key = pd.DataFrame({'Province':['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC'],
                          'Row':[1,1,1,2,2,2,3,3,3],
                          'Col':[1,2,3,1,2,3,1,2,3]})

In [52]:
states_grid = states_rt.join(grid_key.set_index('Province'), on='Province')
states_grid

,Province,Date,Rt,High_90,Low_90,Row,Col
0,EC,2020-04-11,3.84,6.34,0.78,1,1
1,EC,2020-04-12,3.35,5.14,1.27,1,1
2,EC,2020-04-13,3.13,4.52,1.55,1,1
3,EC,2020-04-14,2.72,3.87,1.42,1,1
4,EC,2020-04-15,2.29,3.28,1.14,1,1
...,...,...,...,...,...,...,...
430,WC,2020-06-17,0.54,0.75,0.28,3,3
431,WC,2020-06-18,0.59,0.81,0.33,3,3
432,WC,2020-06-19,0.80,1.02,0.54,3,3
433,WC,2020-06-20,1.00,1.22,0.74,3,3


In [53]:
fig4 = px.line(states_grid, x='Date', y='Rt', color='Province', facet_row='Row', facet_col='Col',
             title='Rt for Covid-19 in South African Provinces')
fig4.update_traces(hovertemplate=None)
fig4.update_layout(hovermode="x")
fig4.show()

In [54]:
fig_px = px.line(states_rt, x='Date', y='Rt', color='Province')
fig_len = len(fig_px['data'])

In [55]:
fig5 = make_subplots(rows=3, cols=3,
                    subplot_titles=state_labels,
                    shared_xaxes=True, shared_yaxes=True)

In [56]:
r = 0
for p in range(fig_len):
    c = (p % 3) + 1
    if (c == 1):
        r+=1
    fig5.add_trace(fig_px['data'][p], row=r, col=c)

In [57]:
fig5.update_layout(title_text="Rt for Covid-19 in South African Provinces", height=700)
fig5.update_traces(hovertemplate=None)
fig5.update_layout(hovermode="x")

# Forecasts

In [58]:
cases_df = cases_series.to_frame()
cases_df = cases_df.reset_index()
cases_df = cases_df.rename(columns={'index':'Date'})
cases_df

,Date,Cases
0,2020-03-05,1
1,2020-03-07,2
2,2020-03-08,3
3,2020-03-09,7
4,2020-03-11,13
...,...,...
102,2020-06-17,80412
103,2020-06-18,83890
104,2020-06-19,87715
105,2020-06-20,92681


In [59]:
from datetime import timedelta, date

In [60]:
f = 30

In [61]:
diff = cases_df['Cases'].diff()

In [62]:
d = diff.values[-1]
d

4621.0

In [63]:
r_scenarios = [1.5, 1.4, 1.3, 1.25, 1.2, 1.15, 1.1, 1.075, 1.05, 1.025, 1.0, 0.975, 0.95, 0.925, 0.9, 0.85, 0.8, 0.7, 0.6, 0.5, 0.25, 0.1]
r_scenarios.append(rt)
r_scenarios.sort(reverse=True)

In [64]:
future_projections = None

for r in r_scenarios:
    projection = cases_df.copy()
    lastd = cases_df['Date'].iloc[-1]
    lastc = cases_df['Cases'].iloc[-1]
    d = diff.values[-1]

    for i in range(f):
        lastd += timedelta(days=1)
        newc = lastc + (d * r)
        d = newc - lastc
        lastc = newc

        calc = pd.DataFrame([[lastd, lastc]], columns=['Date', 'Cases'])
        # TODO: consider concat opertion here for faster processing
        projection = projection.append(calc)
        
    projection['R'] = r
    
    if future_projections is None:
        future_projections = projection
    else:
        future_projections = pd.concat([future_projections, projection])

future_projections

,Date,Cases,R
0,2020-03-05,1.000000,1.5
1,2020-03-07,2.000000,1.5
2,2020-03-08,3.000000,1.5
3,2020-03-09,7.000000,1.5
4,2020-03-11,13.000000,1.5
...,...,...,...
0,2020-07-17,97815.444444,0.1
0,2020-07-18,97815.444444,0.1
0,2020-07-19,97815.444444,0.1
0,2020-07-20,97815.444444,0.1


In [65]:
current_forecast = future_projections.query(f"R == {rt}")
current_forecast

,Date,Cases,R
0,2020-03-05,1.000000e+00,1.24
1,2020-03-07,2.000000e+00,1.24
2,2020-03-08,3.000000e+00,1.24
3,2020-03-09,7.000000e+00,1.24
4,2020-03-11,1.300000e+01,1.24
...,...,...,...
0,2020-07-17,6.484197e+06,1.24
0,2020-07-18,8.022782e+06,1.24
0,2020-07-19,9.930627e+06,1.24
0,2020-07-20,1.229636e+07,1.24


In [80]:
last_forecast = current_forecast.iloc[-1]
future_f = math.trunc(last_forecast['Cases'])
future_f

15229858

In [81]:
infected = future_f / country_pop * 100
f'{infected:.1f}%'

'25.9%'

In [67]:
X0 = current_forecast.iloc[0]['Date']
X1 = last_forecast['Date']
X2 = date.today()

In [68]:
max_forecast = max(current_forecast['Cases']) * 1.05
max_country = country_pop * 1.1
max_future = min(max_forecast, max_country)
max_future

15991351.46241807

In [123]:
fig6 = px.line(current_forecast, x='Date', y='Cases',
               range_y=[0, max_future],
               title='Covid-19 Cases Forecast for Current Rt')
fig6.update_traces(hovertemplate=None)
fig6.update_layout(hovermode="x")

fig6.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X2,
    y0=0,
    x1=X2,
    y1=max_future,
    opacity=0.6,
    line=dict(
        color="Black",
        width=2,
        dash='dashdot'
))

fig6.show()

In [91]:
# pop line template
fig.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X0,
    y0=country_pop,
    x1=X1,
    y1=country_pop,
    opacity=0.6,
    line=dict(
        color="Crimson",
        width=2,
        dash='dash'
))

In [104]:
increasing_forecast = future_projections.query(f"R > 1")

In [111]:
fig7 = px.line(increasing_forecast, x='Date', y='Cases',
               animation_frame='R', height=600, range_y=[0, max_country],
               title='Covid-19 Cases Forecast for Increasing Cases (Rt is bigger than 1)')
fig7.update_layout(hovermode="x")
#fig7["layout"].pop("updatemenus") # to remove play controls

fig7.show()

In [102]:
linear_forecast = future_projections.query(f"R == 1")
max_linear = max(linear_forecast['Cases'])

In [103]:
fig8 = px.line(linear_forecast, x='Date', y='Cases',
               range_y=[0, max_linear],
               title='Covid-19 Cases Forecast for Increasing Cases (Rt is 1)')
fig8.update_traces(hovertemplate=None)
fig8.update_layout(hovermode="x")

fig8.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X2,
    y0=0,
    x1=X2,
    y1=max_future,
    opacity=0.6,
    line=dict(
        color="Black",
        width=2,
        dash='dashdot'
))

fig8.show()

In [121]:
decline_forecast = future_projections.query(f"R < 1")
max_decline = max(decline_forecast['Cases']) * 1.05

In [122]:
fig9 = px.line(decline_forecast, x='Date', y='Cases',
               animation_frame='R', height=600, range_y=[0, max_decline],
               title='Covid-19 Cases Forecast for Decreasing Cases (Rt is less than 1)')
fig9.update_traces(hovertemplate=None)
fig9.update_layout(hovermode="x")

fig9.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X2,
    y0=0,
    x1=X2,
    y1=max_decline,
    opacity=0.6,
    line=dict(
        color="Black",
        width=2,
        dash='dashdot'
))

fig9.show()

# Matplotlib

In [ ]:
repo = 'dsfsi'
url = 'https://raw.githubusercontent.com/' + repo + '/covid19za/master/data/calc/calculated_rt_sa_provincial_cumulative.csv'
states_all_rt_i = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True, index_col=[0,1])

In [ ]:
def plot_rt(result, ax, state_name):
    
    ax.set_title(f"{state_name}")
    
    # Colors
    ABOVE = [1,0,0]
    MIDDLE = [1,1,1]
    BELOW = [0,0,0]
    cmap = ListedColormap(np.r_[
        np.linspace(BELOW,MIDDLE,25),
        np.linspace(MIDDLE,ABOVE,25)
    ])
    color_mapped = lambda y: np.clip(y, .5, 1.5)-.5
    
    index = result['ML'].index.get_level_values('date')
    values = result['ML'].values
    
    # Plot dots and line
    ax.plot(index, values, c='k', zorder=1, alpha=.25)
    ax.scatter(index,
               values,
               s=40,
               lw=.5,
               c=cmap(color_mapped(values)),
               edgecolors='k', zorder=2)
    
    # Aesthetically, extrapolate credible interval by 1 day either side
    lowfn = interp1d(date2num(index),
                     result['Low_90'].values,
                     bounds_error=False,
                     fill_value='extrapolate')
    
    highfn = interp1d(date2num(index),
                      result['High_90'].values,
                      bounds_error=False,
                      fill_value='extrapolate')
    
    extended = pd.date_range(start=pd.Timestamp('2020-03-01'),
                             end=index[-1]+pd.Timedelta(days=1))
    
    ax.fill_between(extended,
                    lowfn(date2num(extended)),
                    highfn(date2num(extended)),
                    color='k',
                    alpha=.1,
                    lw=0,
                    zorder=3)

    ax.axhline(1.0, c='k', lw=1, label='$R_t=1.0$', alpha=.25);
    
    # Formatting
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.DayLocator())
    
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    ax.yaxis.tick_right()
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.margins(0)
    ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
    ax.margins(0)
    ax.set_ylim(0.0, 5.0)
    ax.set_xlim(pd.Timestamp('2020-03-06'), result.index.get_level_values('date')[-1]+pd.Timedelta(days=1))
    #fig.set_facecolor('w')
    
    return ax

In [ ]:
credit = 'Source: covid19trends.co.za - Data: DSFSI'

In [ ]:
country = states_all_rt_i.filter(like='Total RSA', axis=0)
fig_country, ax = plt.subplots(figsize=(600/72,400/72))
ax = plot_rt(country, ax, state_name = '')
ax.set_title(credit, size=12, weight='light')
fig_country.suptitle(f'$R_t$ for COVID-19 in South Africa', size=14)

In [ ]:
states_rt_i = states_all_rt_i.loc[state_filter]

In [ ]:
def all_plot(final_results):
    state_groups = final_results.groupby('state')
    
    ncols = 3
    nrows = int(np.ceil(len(state_groups) / ncols))

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows*3))

    for i, (state_name, result) in enumerate(state_groups):
        axes.flat[i] = plot_rt(result, axes.flat[i], state_name)

    fig.tight_layout()
    fig.set_facecolor('w')
    
    fig.suptitle(f'$R_t$ for COVID-19 in South African Provinces', size=14)
    fig.subplots_adjust(top=0.92)
    
all_plot(states_rt_i)

# Standings

In [ ]:
# ZA: South Arica lockdown level data as of 2020/05/03
no_lockdown = [

]
partial_lockdown = [

]
# add items as required -> 'Western Cape','WC',

FULL_COLOR = [.7,.7,.7]
NONE_COLOR = [179/255,35/255,14/255]
PARTIAL_COLOR = [.5,.5,.5]
ERROR_BAR_COLOR = [.3,.3,.3]

In [ ]:
# ZA: df slighty different to US
mr = states_all_rt_i.groupby(level=0)[['ML', 'High_90', 'Low_90']].last()
mr

In [ ]:
def plot_standings(mr, figsize=None, title='Most Recent $R_t$ by Province'):
    if not figsize:
        figsize = ((15.9/50)*len(mr)+.1,2.5)
        
    fig, ax = plt.subplots(figsize=figsize)

    ax.set_title(title)
    err = mr[['Low_90', 'High_90']].sub(mr['ML'], axis=0).abs()
    bars = ax.bar(mr.index,
                  mr['ML'],
                  width=.825,
                  color=FULL_COLOR,
                  ecolor=ERROR_BAR_COLOR,
                  capsize=2,
                  error_kw={'alpha':.5, 'lw':1},
                  yerr=err.values.T)

    for bar, state_name in zip(bars, mr.index):
        if state_name in no_lockdown:
            bar.set_color(NONE_COLOR)
        if state_name in partial_lockdown:
            bar.set_color(PARTIAL_COLOR)

    labels = mr.index.to_series().replace({'District of Columbia':'DC'})
    ax.set_xticklabels(labels, rotation=90, fontsize=11)
    ax.margins(0)
    ax.set_ylim(0,2.)
    ax.axhline(1.0, linestyle=':', color='k', lw=1)

    leg = ax.legend(handles=[
                        Patch(label='Full', color=FULL_COLOR),
                        Patch(label='Partial', color=PARTIAL_COLOR),
                        Patch(label='None', color=NONE_COLOR)
                    ],
                    title='Lockdown',
                    ncol=3,
                    loc='upper left',
                    columnspacing=.75,
                    handletextpad=.5,
                    handlelength=1)

    leg._legend_box.align = "left"
    fig.set_facecolor('w')
    return fig, ax

mr.sort_values('ML', inplace=True)
plot_standings(mr);

In [ ]:
mr.sort_values('High_90', inplace=True)
plot_standings(mr);

In [ ]:
show = mr[mr.High_90.le(1)].sort_values('ML')
fig, ax = plot_standings(show, title='Likely Under Control');

In [ ]:
show = mr[mr.Low_90.ge(1.0)].sort_values('Low_90')
fig, ax = plot_standings(show, title='Likely Not Under Control');
ax.get_legend().remove()